# Imports

In [54]:
# IMPORTS
import copy
import numpy as np
from enum import Enum

# CLASS INITIALIZATIONS

In [208]:
class Direction(Enum):
  HORIZONTAL = 1
  VERTICAL = 2

class Coordinate(object):
  def __init__(self, X, Y):
    self.X = X 
    self.Y = Y
  # Primarily for testing to see if Coordinate works
  def __str__(self):
    return "(" + str(self.X) + ", " + str(self.Y) + ")"

#
# CAR
# 

class Car(object):
  def __init__(self, letter, coordinate, fuel):
    self.letter = letter  # Remember that A is the most important
    self.coordinates = []
    self.coordinates.append(coordinate)
    self.fuel = fuel

  def isA(self):
    return self.letter == 'A'

  def move(self, amt):
    if self.direction == Direction.HORIZONTAL:
      for c in self.coordinates:
        c.Y += amt
    else:
      for c in self.coordinates:
        c.X += amt
    self.fuel -= 1
  
  def canMove(self):
    return self.fuel > 0

  def compareLetters(self, letter):
    return self.letter == letter

  def addCoordinate(self, newCoord):
    self.coordinates.append(newCoord)
    # Update direction
    # L-R (Horizontal)
    if self.coordinates[0].X == newCoord.X and self.coordinates[0].Y != newCoord.Y:
      self.direction = Direction.HORIZONTAL
    # U-D (Vertical)
    else:
      self.direction = Direction.VERTICAL

  def getLetter(self):
    return self.letter; 

  # Primarily for testing to see if Car works
  def __str__(self):
    s = self.letter + " -> ";
    for i in self.coordinates:
      s += str(i)
    return s + " | " + str(self.fuel) + " | " + str(self.direction)

  @staticmethod
  def create(letter, coordinates, fuel):
    return Car(letter, coordinates, fuel)


#
# BOARD
# 

class Board(object):
  def __init__(self, cars):
    self.grid = [['.' for i in range(6)] for j in range(6)]
    self.size = {'X': 6, 'Y': 6}
    self.cars = []
    for car in cars:  
      c = car
      self.cars.append(copy.deepcopy(c));
      for coord in c.coordinates:
        self.grid[coord.X][coord.Y] = car.letter

  def findCarPaths(self, car):
    fwd = self.checkPathForward(car)
    bwd = self.checkPathBackward(car)
    carPath = fwd + bwd
    return carPath

  def moveCar(self, carLetter, amt):
    for car in self.cars: 
      if car.letter == carLetter:
        for coord in car.coordinates:
          print(coord)
          self.grid[coord.X][coord.Y] = "."
        car.move(amt)
        for coord in car.coordinates:
          print(coord)
          self.grid[coord.X][coord.Y] = carLetter

      

  # ......
  # ...AA. (1,3)(1,4)
  # ......
  # startPoint = 4
  # range(5,6) -> 5
  # next gridPos = (1,5)
  # append -> 5 - 4 = 1

  # ...D.. (0,3)
  # ...D.. (1,3)
  # ......
  # startPoint = 1
  # range(2,6) -> 2, 3, 4, 5
  # next gridPos = (2,3)
  # append -> 2 - 1 = 1
  def checkPathForward(self, car):
    dir = car.direction
    lastCoord = car.coordinates[len(car.coordinates)-1]
    if dir == Direction.HORIZONTAL:
      startPoint = lastCoord.Y
    else:
      startPoint = lastCoord.X

    possibleMoves = []
    for i in range(startPoint+1, 6):
      # Find next position to check
      if dir == Direction.HORIZONTAL:
        gridPos = self.grid[lastCoord.X][i] 
      else:
        gridPos = self.grid[i][lastCoord.Y]

      # If there's an open position, add and continue
      # Else, there is something blocking the way and there's no way to continue any more
      if gridPos == ".":
        possibleMoves.append(i-startPoint)
      else:
        break;

    return possibleMoves

  # ......
  # ...AA. (1,3)(1,4)
  # ......
  # startPoint = 3
  # range(3,0,-1) -> 3, 2, 1
  # i = i - 1 -> 2, 1, 0
  # next gridPos = (1,2)
  # append -> 2 - 3 = -1

  # ...D.. (0,3)
  # ...D.. (1,3)
  # ......
  # startPoint = 0
  # range(0,0) -> NONE
  # next gridPos = NONE
  def checkPathBackward(self, car):
    dir = car.direction
    lastCoord = car.coordinates[0]
    if dir == Direction.HORIZONTAL:
      startPoint = lastCoord.Y
    else:
      startPoint = lastCoord.X

    possibleMoves = []
    for i in range(startPoint, 0, -1):
      # Find next position to check
      if dir == Direction.HORIZONTAL:
        gridPos = self.grid[lastCoord.X][i-1] 
      else:
        gridPos = self.grid[i-1][lastCoord.Y]

      # If there's an open position, add and continue
      # Else, there is something blocking the way and there's no way to continue any more
      if gridPos == ".":
        possibleMoves.append(i-startPoint-1)
      else:
        break;

    return possibleMoves

  def winConditionMet(self):
    if(grid[3][6] == "A"):
      return True
    return False

  def __str__(self): 
    s = ""
    for x in range(6):
      for y in range(6):
        s += self.grid[x][y]
      s += "\n"
      
    return s

# Extracting Data From Sample-input.txt

In [9]:
demos = []
demoFuels = []

with open('sample-input.txt') as f:
    for line in f:
      if not line.startswith(('#', '\n')):
        in_arr = np.array([line.strip()])
        data = np.char.split(in_arr)[0]
        demos.append(data)
f.close()

print(demos)

[['BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL.'], ['..I...BBI.K.GHAAKLGHDDKLG..JEEFF.J..'], ['JBBCCCJDD..MJAAL.MFFKL.N..KGGN.HH...'], ['BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII', 'J0', 'B4'], ['IJBBCCIJDDL.IJAAL.EEK.L...KFF..GGHH.', 'F0', 'G6'], ['BB.G.HE..G.HEAAG.I..FCCIDDF..I..F...']]


# Setting the boards up

In [209]:
boards = []
for data in demos:
  # Seperate Grid Data from the Fuel Parameters
  grid = data[0];
  fuels = []
  if len(data) > 1:
    fuels.append(data[1:])

  cars = []
  for row in range(6):
    for col in range(6):
      current = grid[row*6 + col];
      
      # If Unique: Create a car and append
      # If Not Unique: Concat to same letter
      if current == ".":
        continue

      unique = True
      for c in cars:
        if c.compareLetters(current):
          c.addCoordinate(Coordinate(row,col))
          unique = False

      if unique:
        # Now check if there's a unique fuel amount or if its the default 100
        fuelAmt = 100
        if len(fuels) > 0:
          for f in fuels[0]:
            if current == f[0]:
              fuelAmt = f[1:]

        cars.append(Car.create(current, Coordinate(row,col), fuelAmt));

  # Checking if all cars were made properly
  print("CARS: ")
  print("--------")
  for c in cars:
    print(c)
  print()
  
  board = Board(cars)
  # Checking if all boards were made properly
  print("BOARD: ")
  print("--------")
  print(board)
  print()

  boards.append(board)

CARS: 
--------
B -> (0, 0)(0, 1) | 100 | Direction.HORIZONTAL
I -> (0, 2)(1, 2)(2, 2) | 100 | Direction.VERTICAL
J -> (0, 3)(1, 3) | 100 | Direction.VERTICAL
C -> (1, 4)(1, 5) | 100 | Direction.HORIZONTAL
A -> (2, 3)(2, 4) | 100 | Direction.HORIZONTAL
M -> (2, 5)(3, 5) | 100 | Direction.VERTICAL
G -> (3, 0)(4, 0)(5, 0) | 100 | Direction.VERTICAL
D -> (3, 1)(3, 2) | 100 | Direction.HORIZONTAL
K -> (3, 3)(4, 3) | 100 | Direction.VERTICAL
H -> (4, 1)(5, 1) | 100 | Direction.VERTICAL
L -> (4, 4)(5, 4) | 100 | Direction.VERTICAL
F -> (5, 2)(5, 3) | 100 | Direction.HORIZONTAL

BOARD: 
--------
BBIJ..
..IJCC
..IAAM
GDDK.M
GH.KL.
GHFFL.


CARS: 
--------
I -> (0, 2)(1, 2) | 100 | Direction.VERTICAL
B -> (1, 0)(1, 1) | 100 | Direction.HORIZONTAL
K -> (1, 4)(2, 4)(3, 4) | 100 | Direction.VERTICAL
G -> (2, 0)(3, 0)(4, 0) | 100 | Direction.VERTICAL
H -> (2, 1)(3, 1) | 100 | Direction.VERTICAL
A -> (2, 2)(2, 3) | 100 | Direction.HORIZONTAL
L -> (2, 5)(3, 5) | 100 | Direction.VERTICAL
D -> (3, 2)(3

# Building State Space Search - Uniform Cost Search (UCS)

In [212]:
class Node(object):
  def __init__(self):
    self.cost = 0

class UCS(object):
  def __init__(self):
    self.opened = []
    self.closed = []
    self.numOfSteps = 0

  def search(self, board):
    print(board.grid)
    print(board)

    # TO DO:
    # Look at every car on board
    for c in board.cars:
      # Check which one can move
      if c.fuel <= 0:
        continue

      # Check all possible paths this car can move to (both directions and how far they can move)
      paths = board.findCarPaths(c)
      if len(paths) > 0:  
        print("Car ", c.letter, " can move:", paths)

        for p in paths:
          boardCopy = copy.deepcopy(board)
          boardCopy.moveCar(c.letter, p)
          print(boardCopy)

In [214]:
currentBoard = Board(boards[0].cars)
ucs = UCS()
ucs.search(currentBoard)

[['B', 'B', 'I', 'J', '.', '.'], ['.', '.', 'I', 'J', 'C', 'C'], ['.', '.', 'I', 'A', 'A', 'M'], ['G', 'D', 'D', 'K', '.', 'M'], ['G', 'H', '.', 'K', 'L', '.'], ['G', 'H', 'F', 'F', 'L', '.']]
BBIJ..
..IJCC
..IAAM
GDDK.M
GH.KL.
GHFFL.

Car  M  can move: [1, 2]
(2, 5)
(3, 5)
(3, 5)
(4, 5)
BBIJ..
..IJCC
..IAA.
GDDK.M
GH.KLM
GHFFL.

(2, 5)
(3, 5)
(4, 5)
(5, 5)
BBIJ..
..IJCC
..IAA.
GDDK..
GH.KLM
GHFFLM

Car  G  can move: [-1, -2]
(3, 0)
(4, 0)
(5, 0)
(2, 0)
(3, 0)
(4, 0)
BBIJ..
..IJCC
G.IAAM
GDDK.M
GH.KL.
.HFFL.

(3, 0)
(4, 0)
(5, 0)
(1, 0)
(2, 0)
(3, 0)
BBIJ..
G.IJCC
G.IAAM
GDDK.M
.H.KL.
.HFFL.

Car  L  can move: [-1]
(4, 4)
(5, 4)
(3, 4)
(4, 4)
BBIJ..
..IJCC
..IAAM
GDDKLM
GH.KL.
GHFF..

